In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from datetime import timedelta

class PutOptionStrategy:
    def __init__(self, data, signal_col='signal'):
        """
        Initialize with your prepared dataframe
        Args:
            data: DataFrame with columns ['price', 'volatility', signal_col]
            signal_col: Name of your signal column (1 = buy puts)
        """
        self.data = data.copy()
        self.signal_col = signal_col
        self.risk_free_rate = 0.02  # Annual risk-free rate
        self.trade_log = []
        
    def black_scholes_put(self, S, K, T, sigma, r):
        """Calculate Black-Scholes put option price"""
        d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
        d2 = d1 - sigma*np.sqrt(T)
        put_price = K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
        return put_price

    def calculate_daily_pnl(self, trades):
        """Calculate daily P&L across all option positions"""
        daily_pnl = pd.DataFrame(index=self.data.index, columns=['mtm', 'intrinsic', 'cost_basis'])
        daily_pnl.iloc[:] = 0  # Initialize
        
        for trade in trades:
            # Get trade dates
            entry_idx = self.data.index.get_loc(trade['entry_date'])
            expiry_idx = min(entry_idx + trade['maturity_days'], len(self.data)-1)
            trade_dates = self.data.index[entry_idx:expiry_idx+1]
            
            # Calculate intrinsic values
            strike = trade['strike_price']
            prices = self.data.loc[trade_dates, 'price']
            intrinsic = np.maximum(strike - prices, 0)
            
            # Store daily values
            daily_pnl.loc[trade_dates, 'mtm'] += intrinsic
            daily_pnl.loc[trade_dates, 'intrinsic'] += intrinsic
            daily_pnl.loc[trade_dates, 'cost_basis'] += trade['option_cost']
        
        # Calculate P&L components
        daily_pnl['daily_pnl'] = daily_pnl['mtm'].diff()
        daily_pnl['cumulative_pnl'] = daily_pnl['mtm'] - daily_pnl['cost_basis']
        return daily_pnl

    def run_strategy(self, strike_pct=0.85, maturity_days=126):
        """
        Execute strategy with 6-month puts (126 trading days)
        Handles signal resets according to your specifications
        """
        active_positions = []
        self.trade_log = []
        
        for i, (date, row) in enumerate(self.data.iterrows()):
            # Check if current date is beyond last possible expiry
            if i + maturity_days >= len(self.data):
                continue
            
            current_price = row['price']
            current_vol = row['volatility']
            
            # Handle expiring positions
            active_positions = [
                p for p in active_positions 
                if self.data.index[i + maturity_days] < p['expiry_date']
            ]
            
            # New signal received and no active position
            if row[self.signal_col] == 1 and not active_positions:
                # Calculate option parameters
                strike_price = current_price * strike_pct
                days_to_expiry = maturity_days / 252
                
                # Price the option
                option_cost = self.black_scholes_put(
                    S=current_price,
                    K=strike_price,
                    T=days_to_expiry,
                    sigma=current_vol,
                    r=self.risk_free_rate
                )
                
                # Create new position
                position = {
                    'entry_date': date,
                    'entry_price': current_price,
                    'strike_price': strike_price,
                    'expiry_date': self.data.index[i + maturity_days],
                    'maturity_days': maturity_days,
                    'option_cost': option_cost,
                    'entry_vol': current_vol
                }
                active_positions.append(position)
                self.trade_log.append(position)
        
        # Calculate daily P&L
        daily_results = self.calculate_daily_pnl(self.trade_log)
        
        # Merge with original data
        results = self.data.join(daily_results)
        
        # Calculate final P&L at expiry for each trade
        for trade in self.trade_log:
            expiry_price = results.loc[trade['expiry_date'], 'price']
            trade['expiry_price'] = expiry_price
            trade['final_intrinsic'] = max(trade['strike_price'] - expiry_price, 0)
            trade['net_pnl'] = trade['final_intrinsic'] - trade['option_cost']
        
        return results, pd.DataFrame(self.trade_log)

# Example Usage
if __name__ == "__main__":
    # Load your prepared data (replace this with your actual data loading)
    # Columns needed: ['price', 'volatility', 'signal']
    data = pd.read_csv('your_data.csv', index_col='date', parse_dates=True)
    
    # Initialize and run strategy
    strategy = PutOptionStrategy(data, signal_col='your_signal_column')
    daily_results, trade_results = strategy.run_strategy(strike_pct=0.85)
    
    # Print results
    print("\nTrade Execution Log:")
    print(trade_results[['entry_date', 'expiry_date', 'entry_price', 
                         'strike_price', 'expiry_price', 'final_intrinsic', 
                         'option_cost', 'net_pnl']].to_string())
    
    print("\nDaily P&L Summary:")
    print(daily_results[['price', 'signal', 'mtm', 'daily_pnl', 'cumulative_pnl']].tail(20).to_string())
    
    # Plot cumulative P&L
    plt.figure(figsize=(12, 6))
    daily_results['cumulative_pnl'].plot(title='Cumulative P&L (After Option Costs)')
    plt.ylabel('Profit/Loss')
    plt.grid()
    plt.show()